# Sowing Success: How Machine Learning Helps Farmers Select the Best Crops

![Farmer in a field](farmer_in_a_field.jpg)

Measuring essential soil metrics such as nitrogen, phosphorous, potassium levels, and pH value is an important aspect of assessing soil condition. However, it can be an expensive and time-consuming process, which can cause farmers to prioritize which metrics to measure based on their budget constraints.

Farmers have various options when it comes to deciding which crop to plant each season. Their primary objective is to maximize the yield of their crops, taking into account different factors. One crucial factor that affects crop growth is the condition of the soil in the field, which can be assessed by measuring basic elements such as nitrogen and potassium levels. Each crop has an ideal soil condition that ensures optimal growth and maximum yield.

A farmer reached out to you as a machine learning expert for assistance in selecting the best crop for his field. They've provided you with a dataset called `soil_measures.csv`, which contains:

- `"N"`: Nitrogen content ratio in the soil
- `"P"`: Phosphorous content ratio in the soil
- `"K"`: Potassium content ratio in the soil
- `"pH"` value of the soil
- `"crop"`: categorical values that contain various crops (target variable).

Each row in this dataset represents various measures of the soil in a particular field. Based on these measurements, the crop specified in the `"crop"` column is the optimal choice for that field.  

In this project, you will build multi-class classification models to predict the type of `"crop"` and identify the single most importance feature for predictive performance.

In [1]:
# All required libraries are imported here for you.
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Load the dataset
crops = pd.read_csv("soil_measures.csv")


In [114]:
# get some idea of what data is
print(crops.head())

    N   P   K        ph  crop
0  90  42  43  6.502985  rice
1  85  58  41  7.038096  rice
2  60  55  44  7.840207  rice
3  74  35  40  6.980401  rice
4  78  42  42  7.628473  rice


In [115]:
# checking for nulls - but normally it would include NaNs & ''
empties = crops.isna().sum().sort_values()
print(empties)

N       0
P       0
K       0
ph      0
crop    0
dtype: int64


In [116]:
# gauge ranges of features
print(crops.describe())

                 N            P            K           ph
count  2200.000000  2200.000000  2200.000000  2200.000000
mean     50.551818    53.362727    48.149091     6.469480
std      36.917334    32.985883    50.647931     0.773938
min       0.000000     5.000000     5.000000     3.504752
25%      21.000000    28.000000    20.000000     5.971693
50%      37.000000    51.000000    32.000000     6.425045
75%      84.250000    68.000000    49.000000     6.923643
max     140.000000   145.000000   205.000000     9.935091


So no nulls but the target variable column has to be numeric - BUT logistic regression (suggested model as per intro) can handle categorical variables as `y` for model splitting.

Also, feature variables have wide ranges - so best to standardise them.

In [117]:
# splitting dataset
X = crops.drop("crop", axis=1).values
y = crops["crop"].values

# check shapes
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (2200, 4)
y shape: (2200,)


In [118]:
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train the logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

logreg = model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)

print(logreg.score(X_test, y_test))

0.6840909090909091


Only 68% of model accuracy - so have to tune some hyperparameters. Let's do grid search & randomised search cross-validations, to compare which one is better.

In [119]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold
import numpy as np

# grid search
kf = KFold(n_splits=5, shuffle=True, random_state=42)
params = {'C': np.linspace(0.001, 1, 50),
         'solver': ['lsqr', 'sag', 'newton-cg']}

tuning_grid = GridSearchCV(model, params, cv=kf)

# Fit to find the best model
tuning_grid.fit(X_train, y_train)

print(f"Tuned logreg params: {tuning_grid.best_params_}, accuracy: {tuning_grid.score(X_test, y_test)}, best score: {tuning_grid.best_score_}")

Tuned logreg params: {'C': 0.8572857142857143, 'solver': 'sag'}, accuracy: 0.6863636363636364, best score: 0.6568181818181817


Sounds like grid search cross-validation didn't yield anything - until I used KFold which upped the accuracy marginally (68.64% versus earlier 68.41%). Let's check if randomised search provides about the same results.

In [120]:
# randomised search version
kf = KFold(n_splits=5, shuffle=True, random_state=42)
params = {'C': np.linspace(0.001, 1, 50),
          'penalty': ['l1', 'l2'],
          'tol': np.linspace(0.0001, 1.0, 50),
          'class_weight': ['balanced', {0:0.8, 1:0.2}]}

tuning_rand = RandomizedSearchCV(model, params, cv=kf)

# Fit to find the best model
tuning_rand.fit(X_train, y_train)

print(f"Tuned logreg params: {tuning_rand.best_params_}, accuracy: {tuning_rand.score(X_test, y_test)}, best score: {tuning_rand.best_score_}")
          

Tuned logreg params: {'tol': 0.36741020408163266, 'penalty': 'l2', 'class_weight': 'balanced', 'C': 0.12332653061224491}, accuracy: 0.6431818181818182, best score: 0.6147727272727272


so, same story. Let's do a final thing that I know of: polynomial transformations that **might** help or not.

In [121]:
from sklearn.preprocessing import PolynomialFeatures

# Add polynomial features to dataset
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X_train)    # X_train is now transformed

# ...and a randomised search version again - it's much faster than grid
kf = KFold(n_splits=5, shuffle=True, random_state=42)
params = {'C': np.linspace(0.001, 1, 50),
          'penalty': ['l1', 'l2'],
          'tol': np.linspace(0.0001, 1.0, 50),
          'class_weight': ['balanced', {0:0.8, 1:0.2}]}

tuning_rand = RandomizedSearchCV(model, params, cv=kf)

# Fit to find the best model
tuning_rand.fit(X_train, y_train)

print(f"Tuned logreg params: {tuning_rand.best_params_}, accuracy: {tuning_rand.score(X_test, y_test)}, best score: {tuning_rand.best_score_}")

Tuned logreg params: {'tol': 0.28578571428571425, 'penalty': 'l2', 'class_weight': 'balanced', 'C': 0.8165102040816327}, accuracy: 0.6840909090909091, best score: 0.6607954545454545


Interesting - when rising the degree of polynomial features, accuracy actually drops!!!

However, at the same time the difference between 68.64% and 68.41% isn't that big so let's leave it at that.

Let's now see which feature is a best predictor of the crop.

Since I got rid of column names when creating X and y, I have to use column names from original `crops` dataset, rather than using `X` which only contains values from original dataframe dataset.

In [122]:
print(crops.columns)

Index(['N', 'P', 'K', 'ph', 'crop'], dtype='object')


In [123]:
# Extract the feature names from the original DataFrame
feature_names = crops.columns[:-1]  # Exclude target col

best_score = 0
best_feature = None

# Evaluate each feature individually
for i, feature in enumerate(feature_names):  # Loop over feature names
    # Select the ith feature from scaled data
    X_train_single = X_train[:, i].reshape(-1, 1)
    X_test_single = X_test[:, i].reshape(-1, 1)

    # Train logistic regression on a single feature
    model = LogisticRegression(multi_class='multinomial', solver='sag', max_iter=1000)
    model.fit(X_train_single, y_train)

    # Evaluate accuracy
    score = model.score(X_test_single, y_test)

    if score > best_score:
        best_score = score
        best_feature = feature

# Create dictionary with the best feature and its score
best_predictive_feature = {best_feature: best_score}

print(f"Best predictive feature: {best_feature}")
print(f"Accuracy score: {best_score:.2f}")

Best predictive feature: K
Accuracy score: 0.28


The accuracy of 28% for the best single feature means that Potassium (K) is the strongest individual predictor, but it is not sufficient on its own for robust classification.

The original logistic regression model (68% accuracy) relies on the synergy of all features, which is why it's significantly better.

Since this 28% is lower than the model's 68% accuracy, let's run one other check with dummy values - just to check the baseline accuracy ('random chance'/educated guess) by predicting the most frequent crop in the dataset.

In [124]:
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(strategy="most_frequent")
dummy.fit(X_train, y_train)
baseline_score = dummy.score(X_test, y_test)
print(f"Baseline accuracy (most frequent): {baseline_score:.2f}")


Baseline accuracy (most frequent): 0.05


So this 5% baseline is a sign of class imbalance, meaning some crop types dominate the dataset while others are rare - but is that actually true?

In [125]:
# Count occurrences of each crop type in the original dataset
crop_counts = crops["crop"].value_counts()

# Print the counts
print(crop_counts)

rice           100
maize          100
jute           100
cotton         100
coconut        100
papaya         100
orange         100
apple          100
muskmelon      100
watermelon     100
grapes         100
mango          100
banana         100
pomegranate    100
lentil         100
blackgram      100
mungbean       100
mothbeans      100
pigeonpeas     100
kidneybeans    100
chickpea       100
coffee         100
Name: crop, dtype: int64


...so that's not it - all crop types are of the same quantity. Therefore, there is no point in refitting the model using 'balanced' class weight.